**Codigo que pega as Tarefas da sprint e salva em um csv** 
Necessita de supervisão para logar devido a *captcha e entrar no *WebAdmin

In [1]:
## importações
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [27]:
## Login na Shooting
navegador = webdriver.Chrome()
navegador.get("https://beta.shootinghouse.com.br/login")
login = navegador.find_element('xpath', '//*[@id="mat-input-1"]')
senha = navegador.find_element('xpath','//*[@id="input-password"]')
botao_lg = navegador.find_element('xpath', '//*[@id="login-container"]/div/form-login/div/form/button')
login.send_keys("10796732965")
senha.send_keys("asdpoi@@2022@@$")
botao_lg.click()

In [28]:
## Indo para Tarefas em forma de click
navegador.find_element('xpath', '//*[@id="menu-180"]').click()
navegador.find_element('xpath', '//*[@id="menu-182"]').click() #indo até tarefas

In [29]:
## filtra para a sprint
tipo = navegador.find_element('xpath', '//*[@id="mat-select-value-7"]')
tipo.click()
sprint =  "#158 - 2023/02 DE 16/01/2023 ATÉ 27/01/2023" #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
lista = navegador.find_element('xpath', f'//*[@title="{sprint}"]')
lista.click()
navegador.find_element('xpath', '//*[@id="main-content"]/app-task/div/app-task-filters/div/div/div/form/div/button[1]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="#158 - 2023/02 DE 16/01/2023 ATÉ 27/01/2023"]"}
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x008CF243]
	(No symbol) [0x00857FD1]
	(No symbol) [0x0074D04D]
	(No symbol) [0x0077C0B0]
	(No symbol) [0x0077C22B]
	(No symbol) [0x007AE612]
	(No symbol) [0x007985D4]
	(No symbol) [0x007AC9EB]
	(No symbol) [0x00798386]
	(No symbol) [0x0077163C]
	(No symbol) [0x0077269D]
	GetHandleVerifier [0x00B69A22+2655074]
	GetHandleVerifier [0x00B5CA24+2601828]
	GetHandleVerifier [0x00978C0A+619850]
	GetHandleVerifier [0x00977830+614768]
	(No symbol) [0x008605FC]
	(No symbol) [0x00865968]
	(No symbol) [0x00865A55]
	(No symbol) [0x0087051B]
	BaseThreadInitThunk [0x764D00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E57BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E57B8E+238]


In [30]:
## Pega quantas tarefas e quantas paginas tem 
tarefas = navegador.find_element('xpath', '//*[@id="main-content"]/app-task/div/div/div[2]/h5')
paginas = navegador.find_element('xpath', '//*[@id="main-content"]/app-task/div/div/div[3]/app-pagination/div/ngb-pagination/ul')
quantas = int(paginas.text.replace('\n', '').replace('»', '')[-1])

In [31]:
quantas

8

In [11]:
def pegar_tarefas(lista): 
    ## Essa função é para pegar as tarefas da pagina atual, retornando elas em uma lista de lista
    ## Cada item na lista é uma lista com id, titulo e status da tarefa
    data = []
    organizar = False
    contador = 0
    quantas = int(len(lista) / 5)
    print(f'{quantas} tarefas')
    while organizar == False:
        #             id             titulo               status 
        #print("Tarefa: " + lista[contador][:5])
        orga = [lista[contador][:5], lista[contador][6:], lista[contador+1].replace('        ', '')]
        data.append(orga)
        contador += 5
        if contador == len(lista):
            organizar = True
            print("fim das tarefas da pagina")
            print('-------------------------')
            print(data[-1])
            
    return data

In [53]:
def navegar(quantas):
    
    ## essa funcão serve para caminhar pelas paginas e chamar a funcao que pega as tarefas
    ## Também organiza todas as tarefas em uma unica lista
    data = []
    quantas = quantas + 2  ## Esses valores são devido ao indice do numerodor de paginas 
    print(quantas)
    pg_atual = 3           ## que o valor 1 começa em 3
    ### desbugar #### 
    if quantas > 7:  # clica na 5 e volta pra 1 bug
        pr = '//*[@id="main-content"]/app-task/div/div/div[3]/app-pagination/div/ngb-pagination/ul/li[7]/a'
        navegador.find_element('xpath', pr).click()
        time.sleep(5)
        pr = '//*[@id="main-content"]/app-task/div/div/div[3]/app-pagination/div/ngb-pagination/ul/li[3]/a'
        navegador.find_element('xpath', pr).click()
        time.sleep(7)
    while pg_atual <= quantas:
        print(f"Pegando Tarefas da Pagina: {pg_atual-2}")
        tarefas = navegador.find_element("xpath", '//*[@id="main-content"]/app-task/div/div/div[3]/table/tbody')
        listatf = tarefas.text.split('\n')
        tf = pegar_tarefas(listatf)
        pg_atual += 1
        for i in tf:
            data.append(i)  ## adiciona os itens da lista decebida para a principal
        if pg_atual <= quantas:  ## verifica a pagina pra ir ou não pra proxima
            print('indo para pagina: ', pg_atual-2)
            print('indice: ', pg_atual)
            if pg_atual > 9:
                indice = 9
            else:
                indice = pg_atual
            proxima = '//*[@id="main-content"]/app-task/div/div/div[3]/app-pagination/div/ngb-pagination/ul/li[' + str(indice) + ']'
            botao = navegador.find_element('xpath', proxima)
            #print(proxima)
            botao.click()
            time.sleep(7)
    return data

In [49]:
pr = '//*[@id="main-content"]/app-task/div/div/div[3]/app-pagination/div/ngb-pagination/ul/li[8]/a'
navegador.find_element('xpath', pr).click()

ElementClickInterceptedException: Message: element click intercepted: Element <a tabindex="-1" aria-disabled="true" class="page-link ng-star-inserted">...</a> is not clickable at point (863, 598). Other element would receive the click: <li class="page-item disabled ng-star-inserted" style="">...</li>
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x008CF243]
	(No symbol) [0x00857FD1]
	(No symbol) [0x0074D04D]
	(No symbol) [0x007828B9]
	(No symbol) [0x007808CC]
	(No symbol) [0x0077E4CB]
	(No symbol) [0x0077D164]
	(No symbol) [0x007732A6]
	(No symbol) [0x0079858C]
	(No symbol) [0x00772BFF]
	(No symbol) [0x00798804]
	(No symbol) [0x007AC9EB]
	(No symbol) [0x00798386]
	(No symbol) [0x0077163C]
	(No symbol) [0x0077269D]
	GetHandleVerifier [0x00B69A22+2655074]
	GetHandleVerifier [0x00B5CA24+2601828]
	GetHandleVerifier [0x00978C0A+619850]
	GetHandleVerifier [0x00977830+614768]
	(No symbol) [0x008605FC]
	(No symbol) [0x00865968]
	(No symbol) [0x00865A55]
	(No symbo

In [56]:
tarefas = navegar(quantas) ## Chama a função e guarda na variavel
print(f"Pegou {len(tarefas)} tarefas, sendo a ultima:")
print(tarefas[-1]) ## Retorna a ultima tarefa só pra conferir 

10
Pegando Tarefas da Pagina: 1
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  2
indice:  4
Pegando Tarefas da Pagina: 2
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  3
indice:  5
Pegando Tarefas da Pagina: 3
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  4
indice:  6
Pegando Tarefas da Pagina: 4
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  5
indice:  7
Pegando Tarefas da Pagina: 5
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  6
indice:  8
Pegando Tarefas da Pagina: 6
25 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  7
indice:  9
Pegando Tarefas da Pagina: 7
22 tarefas
fim das tarefas da pagina
-------------------------
indo para pagina:  8
indice:  10
Pegando Tarefas da Pagina: 8
22 tarefas
fim das tarefas da pagina
-------------------------
Pegou 194 tarefas, sendo a ultima:
[

In [52]:
len(tarefas)

194

In [57]:
navegador.close() ## fecha o navegador

In [58]:
import pandas as pd ## Pandas pra organizar 

In [59]:
df = pd.DataFrame(tarefas, columns=['id','titulo','status']) ## Cria o DF
df 

,id,titulo,status
0,15562,MELHORIAS NA LISTAGEM DE TAREFAS NO SHCHAT,AGUARDANDO SPRINT
1,15561,LOJA.RECIFEPISTOL.COM.BR - PEDIDO NÃO APARECE ...,AGUARDANDO CONCLUSÃO
2,15559,CLIENTE NOVO - CBTT.ORG.BR,AGUARDANDO CONCLUSÃO
3,15558,ORDEM DE COMPRAS - CRUDS,AGUARDANDO SPRINT
4,15556,CLUBEDETIROPINHEIRO.COM.BR - APAGAR MENSALIDAD...,AGUARDANDO SPRINT
...,...,...,...
189,9917,TTIROCERTO.COM.BR - AVISO DE CONTAS A PAGAR RE...,AGUARDANDO SPRINT
190,9498,ISTEMA CLIENTE NOVO - REFORMULAÇÃO NO ATENDIME...,AGUARDANDO SPRINT
191,8878,TTF - SHOOTING SCORE - PROGRAMAR MODALIDADE 3GUN,AGUARDANDO SPRINT
192,8472,HOOTINGSCORE - TIFF DE AREA,AGUARDANDO SPRINT


In [60]:
df2 = df.sort_values(['status'], ascending=False) ## Cria outro DF organizado por Status 
df2

,id,titulo,status
21,15505,FGTP.COM.BR - CADASTROS COM A DOCUMENTAÇÃO ENT...,CONCLUIDO
62,15435,CLUBEDETIROCAVEIRA.COM.BR - ERRO AO GERAR DOCU...,CONCLUIDO
45,15465,COMBATE TIRO E CAÇA - TROCA DE MODALIDADE,CONCLUIDO
181,12877,ROS CAMPEONATO ONLINE,CONCLUIDO
104,15316,AGUALIMPA.ORG.BR - CRIAR HOMOLOGAÇÃO BOLETOS S...,CONCLUIDO
...,...,...,...
77,15410,CANCELAR CADASTRO WEB MASTER,AGUARDANDO CONCLUSÃO
76,15412,CANCELAMENTO WEB MASTER,AGUARDANDO CONCLUSÃO
74,15414,CANCELAMENTO CADASTRO WEB MASTER,AGUARDANDO CONCLUSÃO
73,15416,FMGTE.ORG.BR - DELETAR MENSALIDADE EM MASSA GE...,AGUARDANDO CONCLUSÃO


In [61]:
df2.to_excel(r'tarefas2.xlsx', sheet_name='tarefas', index=False) ## Salva em um Arquivo de Excel
df.to_excel(r'tarefas.xlsx', sheet_name='tarefas', index=False) ## Salva em um Arquivo de Excel